In [43]:
#Compare predictions and true labels

import pandas as pd
import json

#pseudocode:

#read in predicted bounding boxes
pred_df = pd.read_csv('/home/cara/oregon_critters/runs/detect/0_mc/pred_boxes_0mc.csv')

#read in true bounding boxes, keep only columns we need, and remove '.JPG' from image name
true_csv = pd.read_csv('/home/cara/oregon_critters/sampled_ds_300_split.csv')
true_df = true_csv[['IName','CName', 'X','Y','W','H', 'cam_type', 'day_night', 'project', 'group']]
true_df['IName'] = true_df.IName.map(lambda x: x.strip('.JPG'))

#rename columns
true_df.rename(columns={"IName": "image_name","CName": "class_true", "X":"x", "Y":"y", "W":"w", "H":"h"}, inplace=True)

/tmp/ipykernel_2887770/1523738177.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_df['IName'] = true_df.IName.map(lambda x: x.strip('.JPG'))
/tmp/ipykernel_2887770/1523738177.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_df.rename(columns={"IName": "image_name","CName": "class_true", "X":"x", "Y":"y", "W":"w", "H":"h"}, inplace=True)


In [44]:
#Need to add image height and width so I can un-normalize the pred bounding boxes

#load json and turn into dataframe
with open('/home/cara/oregon_critters/dataset_300_split_adjXY.json', 'r') as json_file:
    json_data = json.load(json_file)    

json_df = pd.DataFrame(json_data['images'])

#extract just image name from the filename
json_df['image_name'] = json_df['file_name'].str.extract(r'([^/]+)\.JPG')

#merge with pred_df
pred_df = pd.merge(pred_df, json_df[['image_name', 'width', 'height']], left_on='image_name', right_on='image_name', how='left')

#convert box dimensions from normalized 
pred_df['x_pixel'] = pred_df['x'] * pred_df['width']
pred_df['y_pixel'] = pred_df['y'] * pred_df['height']
pred_df['w_pixel'] = pred_df['w'] * pred_df['width']
pred_df['h_pixel'] = pred_df['h'] * pred_df['height']

#now convert from center of box to top left
pred_df['x_coco'] = pred_df['x_pixel'] - (pred_df['w_pixel'] / 2)
pred_df['y_coco'] = pred_df['y_pixel'] - (pred_df['h_pixel'] / 2)


In [ ]:
pred_df.head()

In [49]:
true_df.head()

,image_name,class_true,x,y,w,h,cam_type,day_night,project,group
0,10316-5__2019-08-05__04-43-55(1),Bat,1414.630021,1792.900518,824.016913,425.915281,NaN,night,COA_2019,test
1,10831-1__2019-08-07__01-08-48(7),Bat,107.568710,835.605219,564.228330,486.760322,NaN,night,COA_2019,test
2,8749-3__2019-07-19__00-53-50(1),Bat,972.177590,1817.238534,718.477801,344.788561,NaN,night,COA_2019,train
3,8749-3__2019-07-19__00-53-51(2),Bat,740.803383,774.760179,438.393235,263.661841,NaN,night,COA_2019,train
4,8749-3__2019-07-19__04-01-21(1),Bat,489.133192,1947.041287,422.156448,377.239249,NaN,night,COA_2019,train


In [45]:
# Define function to calculate IoU of two bounding boxes (from Jess):

def calculate_iou(x1, y1, w1, h1, x2, y2, w2, h2):

    # Define bboxes
    box1 = [x1, y1, w1, h1]
    box2 = [x2, y2, w2, h2]

    # Convert bounding box coordinates to the format (x1, y1, x2, y2)
    box1 = [box1[0], box1[1], box1[0] + box1[2], box1[1] + box1[3]]
    box2 = [box2[0], box2[1], box2[0] + box2[2], box2[1] + box2[3]]

    # Calculate the coordinates of the intersection rectangle
    x_intersection = max(box1[0], box2[0])
    y_intersection = max(box1[1], box2[1])
    w_intersection = min(box1[2], box2[2]) - x_intersection
    h_intersection = min(box1[3], box2[3]) - y_intersection

    # Calculate the area of intersection
    area_intersection = max(0, w_intersection) * max(0, h_intersection)

    # Calculate the area of the union
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    area_union = area_box1 + area_box2 - area_intersection

    # Calculate IoU
    iou = area_intersection / area_union

    return iou 


In [ ]:
# Create dataframe to compare all boxes

# box_comparisons = pd.DataFrame(columns=['pred_box', 'pred_image', 'pred_class', 'confidence', 
#                                         'match', 'true_box', 'true_class', 'iou'])
# box_comparisons.head()

In [ ]:
# Now iterate through every combination of predicted/true bounding boxes, calculate IoU, and find matches

thresh = 0.5 #set threshold for IoU

results_list = []

for pred_index, pred_row in pred_df.iterrows(): #iterate over predicted boxes

    pred_image_name = pred_row['image_name']
    matching_rows = true_df[true_df['image_name'] == pred_image_name] #find boxes in matching image
    
    for true_index, true_row in matching_rows.iterrows(): #iterate over true boxes on same image

        #call function to calculate IoU 
        iou_result = calculate_iou(pred_row['x_coco'], pred_row['y_coco'], pred_row['w_pixel'], pred_row['h_pixel'],
                                   true_row['x'], true_row['y'], true_row['w'], true_row['h'])
        
        if iou_result > thresh:
            match = True
            true_class = true_row['class_true']
            
        else: 
            match = False
            true_class = 'background'

        #append a row for this predicted box
        results_list.append({'pred_box': pred_index, 
                             'pred_image': pred_image_name,
                             'pred_class': pred_row['class_pred'],
                             'confidence': pred_row['confidence'], 
                             'match': match, 
                             'true_box': true_index, 
                             'true_class': true_class, 
                             'iou': iou_result})
        
    #break

#print(results_list)

# Create dataframe from the results list
results_df = pd.DataFrame(results_list)

results_df.head()


In [ ]:
# now compare classes for matches

results_df['class_match'] = results_df['pred_class'] == results_df['true_class']

results_df.head()

# Save
results_df.to_csv('/home/cara/oregon_critters/runs/detect/0_mc/pred_comparisions.csv')


In [66]:
# match up cam_type based on image_name


results_df_merged = results_df.merge(true_df[['image_name', 'cam_type', 'project', 'day_night', 'group']], left_on = 'pred_image', right_on = 'image_name', how = 'left')
print(len(results_df))

print(len(results_df_merged))

results_df_merged.head()


3968
8900


,pred_box,pred_image,pred_class,confidence,match,true_box,true_class,iou,class_match,image_name,cam_type,project,day_night,group
0,0,7762-4__2019-08-13__09-17-47(1),Weasel,0.736720,True,10,CanadaJay,0.711232,False,7762-4__2019-08-13__09-17-47(1),NaN,COA_2019,day,val
1,1,7762-4__2019-08-13__09-17-47(1),DarkEyedJunco,0.209668,False,10,background,0.000000,False,7762-4__2019-08-13__09-17-47(1),NaN,COA_2019,day,val
2,2,7762-4__2019-08-13__09-17-47(1),Thrush,0.189620,True,10,CanadaJay,0.736196,False,7762-4__2019-08-13__09-17-47(1),NaN,COA_2019,day,val
3,3,7762-4__2019-08-13__09-17-48(2),Chickadee,0.484186,True,11,CanadaJay,0.511730,False,7762-4__2019-08-13__09-17-48(2),NaN,COA_2019,day,val
4,4,7762-4__2019-08-13__09-17-48(2),DarkEyedJunco,0.186659,False,11,background,0.000000,False,7762-4__2019-08-13__09-17-48(2),NaN,COA_2019,day,val


In [57]:
print(len(results_df.true_box.unique()))

1357


In [69]:
results_df.pred_image.value_counts()


pred_image
Drive_05_05020033                              500
Drive_05_05020031                              256
19102E_trail_0035                              192
IMG_0647                                       135
19106F_bait_0017                                81
                                              ... 
Drive_03_RCNX0090                                1
Drive_05_IMG_0093                                1
Drive_06_05100094_1243                           1
20266-3__20266-3-G__2021-04-28__00-27-13(1)      1
Drive_01_IMG_0089                                1
Name: count, Length: 1276, dtype: int64

In [55]:
#Now get false negatives by extracting indices of all true boxes that weren't matched

false_neg = true_df.query('index not in @results_df.true_box')

false_neg.head(11)

# Save
false_neg.to_csv('/home/cara/oregon_critters/runs/detect/0_mc/pred_false_neg.csv')

